In [ ]:
# ============================================================
# 🎮 Steam Game Recommender System
# Content-Based Filtering using TF-IDF & Cosine Similarity
# ============================================================

# ── STEP 1: Install & Import Libraries ──────────────────────
# kagglehub handles dataset downloading automatically
!pip install kagglehub -q

import kagglehub
import numpy as np
import pandas as pd
import os
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

pd.set_option('display.max_columns', None)
pd.set_option('display.width', 500)
pd.set_option('display.expand_frame_repr', False)

print("✅ Libraries loaded successfully!")


# ── STEP 2: Download & Load the Dataset ─────────────────────
# Uses kagglehub to automatically download the dataset.
# First time running: you'll be prompted to enter your Kaggle credentials.
# After that, the dataset is cached locally — no re-download needed.

# Download latest version of the dataset
path = kagglehub.dataset_download("fronkongames/steam-games-dataset")
print("📦 Path to dataset files:", path)

# Find the CSV inside the downloaded folder
csv_files = [f for f in os.listdir(path) if f.endswith(".csv")]
if not csv_files:
    raise FileNotFoundError("No CSV file found in the downloaded dataset folder.")

CSV_PATH = os.path.join(path, csv_files[0])
print(f"📄 Using file: {csv_files[0]}")

df = pd.read_csv(CSV_PATH)
print(f"✅ Dataset loaded: {df.shape[0]:,} rows × {df.shape[1]} columns")
print(df.head())
print()
df.info()


# ── STEP 3: Preprocess the Data ─────────────────────────────
COLUMNS = ["Name", "About the game", "Tags", "Genres", "Price"]

df_clean = (
    df[COLUMNS]
    .dropna()
    .reset_index(drop=True)
)

print(f"\n✅ After dropping nulls: {len(df_clean):,} games remaining")
print(df_clean.head())


# ── STEP 4: Build the TF-IDF Matrix ─────────────────────────
tfidf = TfidfVectorizer(stop_words="english")
tfidf_matrix = tfidf.fit_transform(df_clean["About the game"])

print(f"\n✅ TF-IDF matrix shape: {tfidf_matrix.shape}")


# ── STEP 5: Compute Cosine Similarity ───────────────────────
cosine_sim = cosine_similarity(tfidf_matrix, tfidf_matrix)

print(f"✅ Cosine similarity matrix shape: {cosine_sim.shape}")


# ── STEP 6: Recommender Function ────────────────────────────
# Build a reverse-lookup Series: game name → DataFrame index
indices = pd.Series(df_clean.index, index=df_clean["Name"])
indices = indices[~indices.index.duplicated(keep="last")]

def steam_game_recommender(title: str, top_n: int = 10) -> pd.DataFrame:
    """
    Recommend games similar to `title` based on description similarity.

    Parameters
    ----------
    title : str
        Exact game name as it appears in the dataset.
    top_n : int
        Number of recommendations to return (default 10).

    Returns
    -------
    pd.DataFrame
        Columns: name, genre, original_price, similarity_score
    """
    if title not in indices:
        close = df_clean[df_clean["Name"].str.contains(title, case=False, na=False)]["Name"].tolist()
        suggestion = f"  Did you mean: {close[:5]}" if close else ""
        raise ValueError(f"'{title}' not found in dataset.{suggestion}")

    game_idx = indices[title]
    scores = pd.Series(cosine_sim[game_idx], name="similarity_score")
    top_indices = scores.sort_values(ascending=False).iloc[1 : top_n + 1].index

    result = df_clean.loc[top_indices, ["Name", "Genres", "Price"]].copy()
    result["similarity_score"] = scores[top_indices].values
    result["similarity_score"] = result["similarity_score"].round(4)
    return result.reset_index(drop=True)


# ── STEP 7: Run Example Recommendations ─────────────────────
test_games = ["DayZ", "Street Fighter V", "DOOM"]

for game in test_games:
    try:
        recs = steam_game_recommender(game, top_n=10)
        print(f"\n{'='*55}")
        print(f"  🎮 Top 10 games similar to: {game}")
        print(f"{'='*55}")
        print(recs.to_string(index=True))
    except ValueError as e:
        print(f"\n⚠️  {e}")


# ── STEP 8: Interactive Query (Optional) ────────────────────
# Uncomment and run in Colab for a live search:
#
# game_title = input("Enter a game title: ")
# try:
#     print(steam_game_recommender(game_title, top_n=10))
# except ValueError as e:
#     print(e)

✅ Libraries loaded successfully!
Using Colab cache for faster access to the 'steam-games-dataset' dataset.
📦 Path to dataset files: /kaggle/input/steam-games-dataset
📄 Using file: games.csv
✅ Dataset loaded: 122,611 rows × 39 columns
                                         AppID          Name Release date  Estimated owners  Peak CCU  Required age  Price  DiscountDLC count                                     About the game                         Supported languages Full audio languages Reviews                                       Header image                                      Website                           Support url                Support email  Windows    Mac  Linux  Metacritic score Metacritic url  User score  Positive  Negative  Score rank  Achievements  Recommendations                                              Notes  Average playtime forever  Average playtime two weeks  Median playtime forever  Median playtime two weeks            Developers            Publishers      